<a href="https://colab.research.google.com/github/purang2/GPT/blob/main/GPT2_inside.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Sequence-to-Sequence Model

## Datasets for Experiments

- CNNDM
- XSum
- SAMSum
- Gigaword

In [1]:
!pip install transformers 
!pip install datasets
!pip install py7zr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 12.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 13.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 KB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.5/94.5 KB 3

In [2]:
from datasets import load_dataset

#dataset_cnndm = load_dataset("cnn_dailymail", '2.0.0') #for summarization

dataset_xsum = load_dataset("xsum")

dataset_xsum
#dataset_cnndm

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "gpt2-xl"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

In [34]:
import pandas as pd 

#input_txt = "Transformer are the"
input_txt = "Fine-Tuning BERT for Quality Evaluation:"
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
iterations = []

#n_steps = 8
n_steps = 250
choices_per_step = 5 

with torch.no_grad():
  for _ in range(n_steps):
    iteration = dict()
    iteration["Input"] = tokenizer.decode(input_ids[0])
    output = model(input_ids = input_ids)
    #첫번째 배치의 마지막 토큰의 로짓을 선택해 소프트맥스를 적용합니다.
    next_token_logits = output.logits[0, -1, :]
    next_token_probs = torch.softmax(next_token_logits, dim=-1)
    sorted_ids = torch.argsort(next_token_probs, dim=-1, descending=True)
    #가장 높은 확률의 토큰을 저장합니다.
    for choice_idx in range(choices_per_step):
      token_id = sorted_ids[choice_idx]
      token_prob  =next_token_probs[token_id].cpu().numpy() 
      token_choice = (
          f"{tokenizer.decode(token_id)} ({100 * token_prob:.2f}%)"
      ) 
      iteration[f"Choice {choice_idx+1}"] = token_choice 
    # 예측한 다음 토큰을 입력에 추가합니다.
    input_ids = torch.cat([input_ids, sorted_ids[None, 0, None]], dim=-1)
    iterations.append(iteration)

pd.DataFrame(iterations)




,Input,Choice 1,Choice 2,Choice 3,Choice 4,Choice 5
0,Fine-Tuning BERT for Quality Evaluation:,\n (25.05%),The (5.66%),A (3.58%),B (2.14%),\n\n (1.36%)
1,Fine-Tuning BERT for Quality Evaluation:\n,\n (99.62%),The (0.04%),I (0.01%),In (0.01%),A (0.01%)
2,Fine-Tuning BERT for Quality Evaluation:\n\n,The (8.64%),B (4.62%),In (2.44%),- (2.40%),We (2.28%)
3,Fine-Tuning BERT for Quality Evaluation:\n\nThe,following (5.00%),B (4.58%),first (1.86%),quality (1.72%),goal (1.67%)
4,Fine-Tuning BERT for Quality Evaluation:\n\nTh...,is (5.95%),are (5.17%),parameters (4.98%),steps (4.28%),table (3.95%)
...,...,...,...,...,...,...
245,Fine-Tuning BERT for Quality Evaluation:\n\nTh...,all (99.96%),the (0.00%),least (0.00%),All (0.00%),All (0.00%)
246,Fine-Tuning BERT for Quality Evaluation:\n\nTh...,. (99.77%),\n (0.05%),", (0.02%)",or (0.01%),.. (0.01%)
247,Fine-Tuning BERT for Quality Evaluation:\n\nTh...,\n (97.79%),\n\n (1.33%),<|endoftext|> (0.69%),( (0.02%),B (0.01%)
248,Fine-Tuning BERT for Quality Evaluation:\n\nTh...,\n (99.96%),. (0.00%),the (0.00%),", (0.00%)",a (0.00%)


In [35]:
model_name_instance = "gpt2-xl"
print(f"[{model_name_instance.upper()}'s OUTPUT]\n{iteration['Input']}")

[GPT2-XL's OUTPUT]
Fine-Tuning BERT for Quality Evaluation:

The following is a list of the most common problems that can occur with the BERT.

1. BERT is not working properly.

2. BERT is not working at all.

3. BERT is not working at all.

4. BERT is not working at all.

5. BERT is not working at all.

6. BERT is not working at all.

7. BERT is not working at all.

8. BERT is not working at all.

9. BERT is not working at all.

10. BERT is not working at all.

11. BERT is not working at all.

12. BERT is not working at all.

13. BERT is not working at all.

14. BERT is not working at all.

15. BERT is not working at all.

16. BERT is not working at all.

17. BERT is not working at all.

18. BERT is not working at all.

19. BERT is not working at all.




### Make it simpler

In [5]:
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
#샘플링을 OFF(=FALSE)하여 앞의 결과를 재현
output = model.generate(input_ids, max_new_tokens=n_steps, do_sample=False)
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Transformer are the most common types of mod. They are


Further Example

- 그리디 디코딩은 패턴이 다소 반복되는 경향을 보임

In [6]:
max_length = 128
input_txt = """In a shocking finding, scientist discovered \
a herd of unicorns living in a remote, previously unexplored \
valley, in the Andes Mountains. Even more surprising to the \
researchers was the fact that the unicorns spoke perfect English. \n\n
"""

input_ids = tokenizer(input_txt, return_tensors ="pt")["input_ids"].to(device)
output_greedy = model.generate(input_ids, max_length=max_length, do_sample=False)

print(tokenizer.decode(output_greedy[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English. 


The researchers, from the University of California, Davis, and the University of Colorado, Boulder, were conducting a study on the Andean cloud forest, which is home to the rare species of cloud forest trees. 


The researchers were conducting a study on the Andean cloud forest, which is home to the rare species of cloud forest trees. 


The researchers were conducting a study on


### Beam Search Decoding

In [7]:
import torch.nn.functional as F

def log_probs_from_logits(logits, labels):
  logp = F.log_softmax(logits, dim=-1)
  logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
  return logp_label

def sequence_logprob(model, labels, input_len=0):
  with torch.no_grad():
    output = model(labels)
    log_probs = log_probs_from_logits(
        output.logits[:,:-1,:],labels[:,1:])
    seq_log_prob = torch.sum(log_probs[:, input_len:])
  return seq_log_prob.cpu().numpy() 

logp = sequence_logprob(model, output_greedy, input_len=len(input_ids[0]))
print(tokenizer.decode(output_greedy[0]))
print(f"\n로그 확률: {logp:.2f}")

In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English. 


The researchers, from the University of California, Davis, and the University of Colorado, Boulder, were conducting a study on the Andean cloud forest, which is home to the rare species of cloud forest trees. 


The researchers were conducting a study on the Andean cloud forest, which is home to the rare species of cloud forest trees. 


The researchers were conducting a study on

로그 확률: -68.37


In [8]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=5, do_sample=False)
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"\n로그 확률: {logp:.2f}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English. 


According to the researchers, the unicorns were found in a remote valley in the Andes Mountains in Peru. The valley is located in a remote area of the Andes Mountains. The valley is located in a remote area of the Andes Mountains. According to the researchers, the unicorns were found in a remote valley in the Andes Mountains in Peru. The valley is located in a remote area

로그 확률: -44.85


it takes about 1 ~ 2 minute.

- 반복적인 텍스트 생성을 막음: log score는 더 낮아졌지만 텍스트가 일관성을 유지한다. 
- 확률이 높은 토큰에 초점을 두는 Beam Search와 반복을 줄이는 N-gram Penalty를 함께 적용하면 아웃풋의 밸런스가 좋다.

In [9]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=5, do_sample=False, no_repeat_ngram_size=2)
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"\n로그 확률: {logp:.2f}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English. 


The researchers, from the University of California, Los Angeles (UCLA) and the Universidad Nacional Autónoma de México (UNAM) in Mexico City, discovered the unicorn herd by accident. They were conducting a study on the effects of climate change on wild animals, when they came across the herd.

"When we first saw them, we couldn't believe our

로그 확률: -77.14


### Sampling

### Top-p sampling : = 'temperature'를 softmax에 부여
- temperature를 완전히 높이니 문장이 엉망진창이 됨. 드문 토크니 강조되어 모델이 이상한 문법, 다양한 가짜 단어를 만들어냄!

In [10]:
output_temp = model.generate(input_ids, max_length = max_length, do_sample = True, temperature=2.0, top_k=0) 

print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English. 


Previously only documented as bull Iowaeni definite Idaho oliveira quarry sol hawiana pen drove be extermination floated ever immensely entertained Jefferson circumstell Palmer Pot Pom saying pillar especially optionally in varying marks condemnation securing eligible Mormons spent rulers Beast faded TNT Olive ram sunset portal statewide accomplishments fadedignty tempor Kau Islam Cruz euphemGW awake animosity apLisa Demon heartbeat Living Metallurgicalscope Fantastic EP Food Country Restore Mostly Weaver Hispanic sisters  Fou


- Temp를 좀 낮추니 일관성이 상승함

In [11]:
output_temp = model.generate(input_ids, max_length = max_length, do_sample = True, temperature=0.5, top_k=0) 

print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English. 


The herd of unicorns are the offspring of a male and a female that was conceived in the wild. The male and female were taken to the valley in the Andes Mountains, and there, the male's sperm fertilized the female's eggs. The female's eggs were then released into the wild. The mother and her offspring then lived in the wild, but they were unable to find a mate


### Top-k Sampling with Top-p

In [12]:
output_temp = model.generate(input_ids, max_length = max_length, do_sample = True, temperature=0.5, top_k=50) 

print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English. 


The scientists were searching for the oldest known species of the unicorn, which was discovered in the Himalayas in the 1960s.

The unicorn was first discovered in the mountains of Tibet in the 1960s and is considered the largest land mammal in the world.

However, the scientists found the unicorn in the remote mountains of the Andes Mountains, in the Andes Mountains, in the


## Let's dive in the new task; Summarization

In [13]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", version="3.0.0")
print(f"특성: {dataset['train'].column_names}")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/default/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

특성: ['article', 'highlights', 'id']


In [14]:
sample = dataset["train"][1]
print(f"""기사 (500개 문자 발췌, 총 길이: {len(sample["article"])})""")
print(sample["article"][:500])
print(f'\n요약 (길이: {len(sample["highlights"])}):')
print(sample["highlights"])

기사 (500개 문자 발췌, 총 길이: 4051)
Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most s

요약 (길이: 281):
Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .


In [15]:
sample_text = dataset["train"][1]["article"][:2000]

summaries = {}

In [16]:
import nltk 
from nltk.tokenize import sent_tokenize 

nltk.download("punkt")

#줄 바꿈 인식시 U.S. 등의 약어로 인해 마침표로 Split 할 수 없는데 NLTK는 이를 해결해 줌
string = "The U.S. are a country. The U.N. is an organization."

sent_tokenize(string)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['The U.S. are a country.', 'The U.N. is an organization.']

gpt2

In [17]:
from transformers import pipeline, set_seed 

set_seed(42)
pipe = pipeline("text-generation", model="gpt2-xl")
gpt2_query = sample_text + "\nTL;DR:\n"
pipe_out = pipe(gpt2_query, max_length=512, clean_up_tokenization_spaces=True)
summaries["gpt2"] = "\n".join(
    sent_tokenize(pipe_out[0]["generated_text"][len(gpt2_query) :])
)

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1186: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [18]:
pipe = pipeline("summarization", model="t5-large")
pipe_out = pipe(sample_text)
summaries["t5"] = "\n".join(
    sent_tokenize(pipe_out[0]["summary_text"])
)

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [19]:
pipe = pipeline("summarization", model="facebook/bart-large-cnn")
pipe_out = pipe(sample_text)
summaries["bart"] = "\n".join(
    sent_tokenize(pipe_out[0]["summary_text"])
)

In [20]:

print("SOURCE")
for i in sent_tokenize(sample_text):
  print(i)
print("\n")

for model_name in summaries: 
  print(model_name.upper())
  print(summaries[model_name])
  print("")

SOURCE
Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.
Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.
MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."
Here, inmates with the most severe mental illnesses are incarcerated until they're ready to appear in court.
Most often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually "avoidable felonies."
He says the arrests often result from confrontations with police.
Mentally ill people often won't do what they're told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, and less likely to fol